In [1]:
import pandas as pand
from requests_html import HTMLSession
from bs4 import BeautifulSoup
from time import sleep
from random import randint

# PART 1

In [2]:
session = HTMLSession()

In [3]:
url = 'https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1675256383'
headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"}

In [4]:
from IPython.core.interactiveshell import softspace
def getdata(url):
    req = session.get(url, headers=headers)
  
    if req.status_code > 500:
        if "To discuss automated access to Amazon data please contact" in req.text:
            print("Page %s was blocked by Amazon.\n"%url)
        else:
            print("Page %s must have been blocked by Amazon as the status code was %d"%(url,req.status_code))
        return None
    soup = BeautifulSoup(req.html.html, 'html.parser')
    return soup

In [5]:
def getNextPage(soup):
    page = soup.find('span', {'class':'s-pagination-strip'})
    if not page.find('a', {'class':'s-pagination-item s-pagination-next s-pagination-disabled '}):
        link = page.find('a', {'class':'s-pagination-item s-pagination-next s-pagination-button s-pagination-separator'})
        if not link:
            return 
    url = 'http://www.amazon.in' + str(link['href'])
    return url
    else:
        return

In [6]:
pageUrls = []

In [7]:
while True:
    pageUrls.append(url)
    soup = getdata(url)
    url = getNextPage(soup)
    if not url:
        break

In [8]:
pageUrls

['https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1675256383',
 'http://www.amazon.in/s?k=bags&page=2&crid=2M096C61O4MLT&qid=1675344756&ref=sr_pg_1',
 'http://www.amazon.in/s?k=bags&page=3&crid=2M096C61O4MLT&qid=1675344757&ref=sr_pg_2',
 'http://www.amazon.in/s?k=bags&page=4&crid=2M096C61O4MLT&qid=1675344758&ref=sr_pg_3',
 'http://www.amazon.in/s?k=bags&page=5&crid=2M096C61O4MLT&qid=1675344759&ref=sr_pg_4',
 'http://www.amazon.in/s?k=bags&page=6&crid=2M096C61O4MLT&qid=1675344760&ref=sr_pg_5',
 'http://www.amazon.in/s?k=bags&page=7&crid=2M096C61O4MLT&qid=1675344761&ref=sr_pg_6',
 'http://www.amazon.in/s?k=bags&page=8&crid=2M096C61O4MLT&qid=1675344762&ref=sr_pg_7',
 'http://www.amazon.in/s?k=bags&page=9&crid=2M096C61O4MLT&qid=1675344764&ref=sr_pg_8',
 'http://www.amazon.in/s?k=bags&page=10&crid=2M096C61O4MLT&qid=1675344765&ref=sr_pg_9',
 'http://www.amazon.in/s?k=bags&page=11&crid=2M096C61O4MLT&qid=1675344766&ref=sr_pg_10',
 'http://www.amazon.in/s?k=bags&page=12&crid=2M096C61O4MLT

In [9]:
data = []

In [10]:
def appendInfo(content):
    for item in content:
        link = item.find('a', {'class':'a-link-normal s-no-outline'})
        url = 'https://www.amazon.in' + str(link['href'])
        name = item.find('span', {'class':'a-size-medium a-color-base a-text-normal'})
        price = item.find('span', {'class':'a-offscreen'})
        rating = item.find('span', {'class':'a-icon-alt'})
        review = item.find('span', {'class':'a-size-base s-underline-text'})
        if not rating:
            rating = "-"
            review = "-"
        else:
            rating = rating.text
            review = review.text
            review = str(review)
            review = review.replace("(", "")
            review = review.replace(")", "")
        data.append({
            'Product URL': url,
            'Product Name': name.text,
            'Product Price': price.text,
            'Rating': rating,
            'Reviews': review
        })

In [11]:
for page in pageUrls:
    soup = getdata(page)
    content = soup.find_all('div', {'class':'sg-col-20-of-24 s-result-item s-asin sg-col-0-of-12 sg-col-16-of-20 sg-col s-widget-spacing-small sg-col-12-of-16'})
    appendInfo(content)

In [12]:
len(data)

306

In [13]:
data = data[0:200]

In [14]:
data = pand.DataFrame(data)

In [15]:
data

,Product URL,Product Name,Product Price,Rating,Reviews
0,https://www.amazon.in/American-Tourister-AMT-S...,American Tourister 32 Ltrs Black Casual Backpa...,"₹1,299",4.1 out of 5 stars,"51,363"
1,https://www.amazon.in/Wesley-Milestone-Waterpr...,Wesley Milestone 2.0 Casual Waterproof Laptop ...,₹565,4.3 out of 5 stars,"9,224"
2,https://www.amazon.in/Skybags-Brat-Black-Casua...,Skybags Brat Black 46 Cms Casual Backpack,₹669,4.1 out of 5 stars,"3,365"
3,https://www.amazon.in/ADISA-Laptop-Backpack-Of...,ADISA 15.6 inch Laptop Backpack Office Bag Col...,₹499,3.9 out of 5 stars,461
4,https://www.amazon.in/AirCase-C34-Laptop-Backp...,AirCase Laptop Bag Backpack Case Pouch for Lap...,₹999,3.9 out of 5 stars,709
...,...,...,...,...,...
195,https://www.amazon.in/POLESTAR-backpack-Backpa...,"POLESTAR Leader 34 L Bag for College, School, ...",₹599,4.1 out of 5 stars,"3,400"
196,https://www.amazon.in/Artistix-Backpack-Anti-T...,Artistix 25 ltrs (30 cms)backpack(Talon_40cm_b...,₹999,4.3 out of 5 stars,"2,763"
197,https://www.amazon.in/Gear-Black-Casual-Backpa...,Gear Campus1 22L Water Resistant School Bag/ C...,₹649,4.3 out of 5 stars,"3,831"
198,https://www.amazon.in/Wooum-Resisitance-Should...,wooum Water Resisitance Sling Cross Body Bag/O...,₹328.70,4.0 out of 5 stars,575


# PART 2

In [16]:
links = data["Product URL"]

In [17]:
descriptions = []
asins = []
manufacturers = []
prodDescs = []

In [18]:
def clean(item):
    item = item.replace("  ", "")
    item = item.replace("\n", "")
    item = item.replace("\u200e", "")
    return item

In [19]:
def findASIN(soup):
    content = soup.find('div', {'class':'a-column a-span6 a-span-last'})
    if not content:
        asin = "-"
    else:
        head = soup.find_all('th', {'class':'a-color-secondary a-size-base prodDetSectionEntry'})
        value = soup.find_all('td', {'class':'a-size-base prodDetAttrValue'})
        for i in range(len(head)):
            if head[i].text == " ASIN ":
                asin = value[i].text
                asin = clean(asin)
    asins.append(asin)

In [20]:
def findManufacturer(soup):
    content = soup.find('div', {'class':'a-column a-span6 a-span-last'})
    manufacturer = "-"
    if not content:
        manufacturer = "-"
    else:
        head = soup.find_all('th', {'class':'a-color-secondary a-size-base prodDetSectionEntry'})
        value = soup.find_all('td', {'class':'a-size-base prodDetAttrValue'})
        for i in range(len(head)):
            if head[i].text == " Manufacturer ":
                manufacturer = value[i].text
                manufacturer = clean(manufacturer)
                break
        manufacturers.append(manufacturer)

In [21]:
def getAllInfo(soup):
    description = soup.find('ul', {'class':'a-unordered-list a-vertical a-spacing-mini'})
    if not description:
        description = "-"
    else:
        description = description.text
    descriptions.append(description)
    findASIN(soup)
    findManufacturer(soup)
    content = soup.find('table', {'id':'productDetails_techSpec_section_1'})
    if not content:
        desc = "-"
    else:
        prodDesc = content.find_all('td', {'class':'a-size-base prodDetAttrValue'})
        desc = ""
        for item in prodDesc:
            item = str(item.text)
            item = item.replace(" ", "")
            desc += item + " "
            desc = clean(desc)
    prodDescs.append(desc) 

In [22]:
i = 1
for url in links:
    soup = getdata(url)
    sleep(randint(2,10))
    print("Product: ", i)
    i+= 1
    getAllInfo(soup)

Product:  1
Product:  2
Product:  3
Product:  4
Product:  5
Product:  6
Product:  7
Product:  8
Product:  9
Product:  10
Product:  11
Product:  12
Product:  13
Product:  14
Product:  15
Product:  16
Product:  17
Product:  18
Product:  19
Product:  20
Product:  21
Product:  22
Product:  23
Product:  24
Product:  25
Product:  26
Product:  27
Product:  28
Product:  29
Product:  30
Product:  31
Product:  32
Product:  33
Product:  34
Product:  35
Product:  36
Product:  37
Product:  38
Product:  39
Product:  40
Product:  41
Product:  42
Product:  43
Product:  44
Product:  45
Product:  46
Product:  47
Product:  48
Product:  49
Product:  50
Product:  51
Product:  52
Product:  53
Product:  54
Product:  55
Product:  56
Product:  57
Product:  58
Product:  59
Product:  60
Product:  61
Product:  62
Product:  63
Product:  64
Product:  65
Product:  66
Product:  67
Product:  68
Product:  69
Product:  70
Product:  71
Product:  72
Product:  73
Product:  74
Product:  75
Product:  76
Product:  77
Product:

In [23]:
data['Description'] = descriptions
data['ASIN'] = asins
data['Manufacturer'] = manufacturers
data['Product Description'] = prodDescs

In [24]:
data

,Product URL,Product Name,Product Price,Rating,Reviews,Description,ASIN,Manufacturer,Product Description
0,https://www.amazon.in/American-Tourister-AMT-S...,American Tourister 32 Ltrs Black Casual Backpa...,"₹1,299",4.1 out of 5 stars,"51,363","Laptop Compatibility: No, Strap Type: Adjust...",B07CJCGM1M,Samsonite,Black-Grey 0.5Kilograms AMTFIZZSCHBAG02-BLACK ...
1,https://www.amazon.in/Wesley-Milestone-Waterpr...,Wesley Milestone 2.0 Casual Waterproof Laptop ...,₹565,4.3 out of 5 stars,"9,224",30L Capacity: The Backpack has a padded lapt...,B084JGJ8PF,"Wesley, Longani Trading Company, F-82, Sector-...","Wesley Wesley,LonganiTradingCompany,F-82,Secto..."
2,https://www.amazon.in/Skybags-Brat-Black-Casua...,Skybags Brat Black 46 Cms Casual Backpack,₹669,4.1 out of 5 stars,"3,365",Combination of functional & safety features ...,-,-,-
3,https://www.amazon.in/ADISA-Laptop-Backpack-Of...,ADISA 15.6 inch Laptop Backpack Office Bag Col...,₹499,3.9 out of 5 stars,461,Material: Water Resistant Light-Weight Polye...,-,-,-
4,https://www.amazon.in/AirCase-C34-Laptop-Backp...,AirCase Laptop Bag Backpack Case Pouch for Lap...,₹999,3.9 out of 5 stars,709,Innovative Design: AirCase Laptop Backpack i...,-,-,-
...,...,...,...,...,...,...,...,...,...
195,https://www.amazon.in/POLESTAR-backpack-Backpa...,"POLESTAR Leader 34 L Bag for College, School, ...",₹599,4.1 out of 5 stars,"3,400",PADDED LAPTOP COMPARTMENT WITH MAXED OUT STO...,-,-,-
196,https://www.amazon.in/Artistix-Backpack-Anti-T...,Artistix 25 ltrs (30 cms)backpack(Talon_40cm_b...,₹999,4.3 out of 5 stars,"2,763",Fit Type: Western MATERIAL: High Density ...,-,-,-
197,https://www.amazon.in/Gear-Black-Casual-Backpa...,Gear Campus1 22L Water Resistant School Bag/ C...,₹649,4.3 out of 5 stars,"3,831","Outer Material: Polyester, Inner Material: P...",-,-,-
198,https://www.amazon.in/Wooum-Resisitance-Should...,wooum Water Resisitance Sling Cross Body Bag/O...,₹328.70,4.0 out of 5 stars,575,Crossbody Travel Pouch comes with 4 Zip Pock...,-,-,-


In [25]:
data.to_csv("products.csv")